<a href="https://colab.research.google.com/github/DorakuZz/people_counting/blob/main/PeopleCounting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Asegúrate de usar las bibliotecas necesarias
pip install ultralytics opencv-python-headless matplotlib


In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Cargar el modelo preentrenado YOLOv8
model = YOLO('yolov8n.pt')  # Puedes usar 'yolov8m.pt' para un modelo más preciso

# Función para detección de personas
def detect_people(frame):
    results = model(frame)  # Procesar el frame con YOLO
    detections = results[0].boxes.xyxy  # Coordenadas de las detecciones
    classes = results[0].boxes.cls  # Clases detectadas
    confidences = results[0].boxes.conf  # Confianzas

    people_count = 0
    for i, cls in enumerate(classes):
        if int(cls) == 0:  # Clase 0 en YOLO corresponde a "persona"
            people_count += 1
            x1, y1, x2, y2 = map(int, detections[i])
            confidence = confidences[i]
            # Dibujar un rectángulo alrededor de la persona
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f'Person: {confidence:.2f}', (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    return frame, people_count


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 79.2MB/s]


In [ ]:
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame, count = detect_people(frame)

        # Mostrar el frame procesado
        cv2.putText(frame, f'People Count: {count}', (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.imshow('People Counting', frame)

        # Salir con 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [ ]:
from google.colab.patches import cv2_imshow  # Importar cv2_imshow para Colab

def process_video_colab(video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Detectar personas en el cuadro
        frame, count = detect_people(frame)

        # Añadir texto con el conteo de personas
        cv2.putText(frame, f'People Count: {count}', (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Mostrar el cuadro procesado en Colab
        cv2_imshow(frame)

        # Salir después de mostrar el cuadro (opcional para evitar la visualización continua)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()



In [ ]:
from google.colab import files
uploaded = files.upload()  # Sube un archivo de video

video_path = list(uploaded.keys())[0]
process_video_colab(video_path)


In [ ]:
def save_processed_video(video_path, output_path='output.avi'):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame, _ = detect_people(frame)
        out.write(frame)

    cap.release()
    out.release()


In [ ]:
import cv2

def process_live_camera():
    # Abrir la cámara (0 es generalmente la cámara principal)
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("No se pudo acceder a la cámara.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            print("No se pudo capturar el cuadro.")
            break

        # Detectar personas en el cuadro
        frame, count = detect_people(frame)

        # Mostrar el conteo de personas
        cv2.putText(frame, f'People Count: {count}', (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Mostrar el cuadro procesado en una ventana
        cv2.imshow('Live Camera - People Counting', frame)

        # Presiona 'q' para salir
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
